In [ ]:
# Copyright 2021 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# E2E ML on GCP: MLOps stage 3 : formalization: get started with AutoML pipeline components

<table align="left">
  <td>
    <a href="https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/community/ml_ops/stage3/get_started_with_automl_pipeline_components.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      View on GitHub
    </a>
  </td>
  <td>
    <a href="https://console.cloud.google.com/ai/platform/notebooks/deploy-notebook?download_url=https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/community/ml_ops/stage3/get_started_with_automl_pipeline_components.ipynb">
      Open in Google Cloud Notebooks
    </a>
  </td>
</table>
<br/><br/><br/>

## Overview


This tutorial demonstrates how to use Vertex AI for E2E MLOps on Google Cloud in production. This tutorial covers stage 3 : formalization: get started with AutoML pipeline components.

### Dataset

The dataset used for this tutorial is the [Flowers dataset](https://www.tensorflow.org/datasets/catalog/tf_flowers) from [TensorFlow Datasets](https://www.tensorflow.org/datasets/catalog/overview). The version of the dataset you will use in this tutorial is stored in a public Cloud Storage bucket. The trained model predicts the type of flower an image is from a class of five flowers: daisy, dandelion, rose, sunflower, or tulip.

### Objective

In this tutorial, you learn how to use prebuilt `Google Cloud Pipeline Components` for `Vertex AI AutoML`.

This tutorial uses the following Google Cloud ML services:

- `Vertex AI Pipelines`
- `Vertex AI AutoML`
- `Google Cloud Pipeline Components`
- `Vertex AI Dataset, Model and Endpoint` resources

The steps performed include:

- Construct a pipeline for training and deploying a Vertex AI AutoML model.
- Execute a Vertex AI pipeline.

## Installations

Install *one time* the packages for executing the MLOps notebooks.

In [ ]:
ONCE_ONLY = False
if ONCE_ONLY:
    ! pip3 install -U tensorflow==2.5 $USER_FLAG
    ! pip3 install -U tensorflow-data-validation==1.2 $USER_FLAG
    ! pip3 install -U tensorflow-transform==1.2 $USER_FLAG
    ! pip3 install -U tensorflow-io==0.18 $USER_FLAG
    ! pip3 install --upgrade google-cloud-aiplatform[tensorboard] $USER_FLAG
    ! pip3 install --upgrade google-cloud-pipeline-components $USER_FLAG
    ! pip3 install --upgrade google-cloud-bigquery $USER_FLAG
    ! pip3 install --upgrade google-cloud-logging $USER_FLAG
    ! pip3 install --upgrade apache-beam[gcp] $USER_FLAG
    ! pip3 install --upgrade pyarrow $USER_FLAG
    ! pip3 install --upgrade cloudml-hypertune $USER_FLAG
    ! pip3 install --upgrade kfp $USER_FLAG

### Restart the kernel

Once you've installed the additional packages, you need to restart the notebook kernel so it can find the packages.

In [ ]:
import os

if not os.getenv("IS_TESTING"):
    # Automatically restart kernel after installs
    import IPython
    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

#### Set your project ID

**If you don't know your project ID**, you may be able to get your project ID using `gcloud`.

In [ ]:
PROJECT_ID = "[your-project-id]"  # @param {type:"string"}

In [ ]:
if PROJECT_ID == "" or PROJECT_ID is None or PROJECT_ID == "[your-project-id]":
    # Get your GCP project id from gcloud
    shell_output = ! gcloud config list --format 'value(core.project)' 2>/dev/null
    PROJECT_ID = shell_output[0]
    print("Project ID:", PROJECT_ID)

In [ ]:
! gcloud config set project $PROJECT_ID

#### Region

You can also change the `REGION` variable, which is used for operations
throughout the rest of this notebook.  Below are regions supported for Vertex AI. We recommend that you choose the region closest to you.

- Americas: `us-central1`
- Europe: `europe-west4`
- Asia Pacific: `asia-east1`

You may not use a multi-regional bucket for training with Vertex AI. Not all regions provide support for all Vertex AI services.

Learn more about [Vertex AI regions](https://cloud.google.com/vertex-ai/docs/general/locations).

In [ ]:
REGION = 'us-central1'  # @param {type: "string"}

#### Timestamp

If you are in a live tutorial session, you might be using a shared test account or project. To avoid name collisions between users on resources created, you create a timestamp for each instance session, and append the timestamp onto the name of resources you create in this tutorial.

In [ ]:
from datetime import datetime

TIMESTAMP = datetime.now().strftime("%Y%m%d%H%M%S")

### Create a Cloud Storage bucket

**The following steps are required, regardless of your notebook environment.**

When you initialize the Vertex SDK for Python, you specify a Cloud Storage staging bucket. The staging bucket is where all the data associated with your dataset and model resources are retained across sessions.

Set the name of your Cloud Storage bucket below. Bucket names must be globally unique across all Google Cloud projects, including those outside of your organization.

In [ ]:
BUCKET_NAME = "gs://[your-bucket-name]"  # @param {type:"string"}

In [ ]:
if BUCKET_NAME == "" or BUCKET_NAME is None or BUCKET_NAME == "gs://[your-bucket-name]":
    BUCKET_NAME = "gs://" + PROJECT_ID + "aip-" + TIMESTAMP

**Only if your bucket doesn't already exist**: Run the following cell to create your Cloud Storage bucket.

In [ ]:
! gsutil mb -l $REGION $BUCKET_NAME

Finally, validate access to your Cloud Storage bucket by examining its contents:

In [ ]:
! gsutil ls -al $BUCKET_NAME

#### Service Account

**If you don't know your service account**, try to get your service account using `gcloud` command by executing the second cell below.

In [ ]:
SERVICE_ACCOUNT = "[your-service-account]"  # @param {type:"string"}

In [ ]:
if SERVICE_ACCOUNT == "" or SERVICE_ACCOUNT is None or SERVICE_ACCOUNT == "[your-service-account]":
    # Get your GCP project id from gcloud
    shell_output = !gcloud auth list 2>/dev/null
    SERVICE_ACCOUNT = shell_output[2].strip()
    print("Service Account:", SERVICE_ACCOUNT)

#### Set service account access for Vertex AI Pipelines

Run the following commands to grant your service account access to read and write pipeline artifacts in the bucket that you created in the previous step -- you only need to run these once per service account.

In [ ]:
! gsutil iam ch serviceAccount:{SERVICE_ACCOUNT}:roles/storage.objectCreator $BUCKET_NAME

! gsutil iam ch serviceAccount:{SERVICE_ACCOUNT}:roles/storage.objectViewer $BUCKET_NAME

### Set up variables

Next, set up some variables used throughout the tutorial.
### Import libraries and define constants

In [ ]:
import google.cloud.aiplatform as aip

#### Import TensorFlow

Import the TensorFlow package into your Python environment.

In [ ]:
import tensorflow as tf

In [ ]:
import json

from kfp import dsl
from kfp.v2 import compiler
from kfp.v2.dsl import component

### Initialize Vertex AI SDK for Python

Initialize the Vertex AI SDK for Python for your project and corresponding bucket.

In [ ]:
aip.init(project=PROJECT_ID, location=REGION, staging_bucket=BUCKET_NAME)

#### Location of Cloud Storage training data.

Now set the variable `IMPORT_FILE` to the location of the CSV index file in Cloud Storage.

In [ ]:
IMPORT_FILE = 'gs://cloud-samples-data/vision/automl_classification/flowers/all_data_v2.csv'

### Create AutoML model evaluation component

The Vertex AI pre-built pipeline components does not currently have a component for retrieiving the model evaluations for a AutoML model. So, you will first write your own component, as follows:

- Takes as input the region and Model artifacts returned from an AutoML training component.
- Create a client interface to the Vertex AI Model service (`metadata["resource_name"]).
- Construct the resource ID for the model from the model artifact parameter.
- Retrieve the model evaluation
- Return the model evaluation as a string.

In [ ]:
from kfp.v2.dsl import Input, Artifact, Model

@component(packages_to_install=["google-cloud-aiplatform"])
def evaluateAutoMLModelOp(model: Input[Artifact],
                          region: str) -> str:
    import google.cloud.aiplatform.gapic as gapic
    import logging


    # Get a reference to the Model Service client
    client_options = {"api_endpoint": f"{region}-aiplatform.googleapis.com"}
    model_service_client = gapic.ModelServiceClient(
        client_options=client_options
    )

    model_id = model.metadata["resourceName"]

    model_evaluations = model_service_client.list_model_evaluations(parent=model_id)
    model_evaluation = list(model_evaluations)[0]
    logging.info(model_evaluation)
    return str(model_evaluation)

## Construct AutoML training pipeline

In the example below, you construct a pipeline for training an AutoML model using pre-built Google Cloud Pipeline Components for AutoML, as follows:

1. Use the prebuilt component `ImageDatasetCreateOp` to create a Vertex AI Dataset resource, where:
    - The display name for the dataset is passed into the pipeline.
    - The import file for the dataset is passed into the pipeline.
    - The component returns the dataset resource as `outputs["dataset"]`
2. Use the prebuilt component `AutoMLImageTrainingJobRunOp` to train a Vertex AI AutoML Model resource, where:
    - The display name for the dataset is passed into the pipeline.
    - The dataset is the output from the `ImageDatasetCreateOp`.
    - The component returns the model resource as `outputs["model"]`.
3. Use the prebuilt component `EndpointCreateOp` to create a Vertex AI Endpoint to deploy the trained model to, where:
    - Since the component has no dependencies on other components, by default it would be executed in parallel with the model training.
    - The `after(training_op)` is added to serialize its execution, so its only executed if the training operation completes successfully.
    - The component returns the endpoint resource as `outputs["endpoint"]`.
4. Use the prebuilt component `ModelDeployOp` to deploy the trained AutoML model to, where:
    - The display name for the dataset is passed into the pipeline.
    - The model is the output from the `AutoMLTrainingJobRunOp`.
    - The endpoint is the output from the `EndpointCreateOp`

*Note:* Since each component is executed as a graph node in its own execution context, you pass the parameter `project` for each component op, in constrast to doing a `aip.init(project=project)` if this was a Python script calling the SDK methods directly within the same execution context.

In [ ]:
from google_cloud_pipeline_components import aiplatform as gcc_aip

PIPELINE_ROOT = "{}/pipeline_root/automl_icn_training".format(BUCKET_NAME)

@dsl.pipeline(name="automl-icn-training",
              description="AutoML image classification training"
             )
def pipeline(import_file: str,
             display_name: str,
             project: str = PROJECT_ID,
             region: str = REGION):

    dataset_op = gcc_aip.ImageDatasetCreateOp(
        project=project,
        display_name=display_name,
        gcs_source=import_file,
        import_schema_uri=aip.schema.dataset.ioformat.image.single_label_classification,
    )

    training_op = gcc_aip.AutoMLImageTrainingJobRunOp(
        project=project,
        display_name=display_name,
        prediction_type="classification",
        model_type="CLOUD",
        base_model=None,
        dataset=dataset_op.outputs["dataset"],
        model_display_name=display_name,
        training_fraction_split=0.6,
        validation_fraction_split=0.2,
        test_fraction_split=0.2,
        budget_milli_node_hours=8000,
    )

    eval_op = evaluateAutoMLModelOp(model=training_op.outputs["model"],
                                    region=region)

    endpoint_op = gcc_aip.EndpointCreateOp(
        project=project,
        location=region,
        display_name=display_name,
    ).after(eval_op)

    deploy_op = gcc_aip.ModelDeployOp(
        model=training_op.outputs["model"],
        endpoint=endpoint_op.outputs["endpoint"],
        automatic_resources_min_replica_count=1,
        automatic_resources_max_replica_count=1,
    )


### Compile and execute the pipeline

Next, you compile the pipeline and then exeute it. The pipeline takes the following parameters, which are passed as the dictionary `parameter_values`:

- `import_file`: The Cloud Storage path to the dataset index file.
- `display_name`: The display name for the generated Vertex AI resources.
- `project`: The project ID.
- `region`: The region.

In [ ]:
compiler.Compiler().compile(
    pipeline_func=pipeline, package_path="automl_icn_training.json"
)

pipeline = aip.PipelineJob(
    display_name="automl_icn_training",
    template_path="automl_icn_training.json",
    pipeline_root=PIPELINE_ROOT,
    parameter_values = { 'import_file': IMPORT_FILE,
                         'display_name': "flowers" + TIMESTAMP,
                         'project': PROJECT_ID,
                         'region': REGION
                       }

)

pipeline.run()

! rm -f automl_icn_training.json

### View AutoML training pipeline results

Finally, you will view the artifact outputs of each task in the pipeline.

In [ ]:
PROJECT_NUMBER = pipeline.gca_resource.name.split('/')[1]
print(PROJECT_NUMBER)

def print_pipeline_output(job, output_task_name):
    JOB_ID = job.name
    print(JOB_ID)
    for _ in range(len(job.gca_resource.job_detail.task_details)):
        TASK_ID = job.gca_resource.job_detail.task_details[_].task_id
        EXECUTE_OUTPUT = PIPELINE_ROOT + '/' + PROJECT_NUMBER + '/' + JOB_ID + '/' + output_task_name + '_' + str(TASK_ID) + '/executor_output.json'
        GCP_RESOURCES = PIPELINE_ROOT + '/' + PROJECT_NUMBER + '/' + JOB_ID + '/' + output_task_name + '_' + str(TASK_ID) + '/gcp_resources'
        if tf.io.gfile.exists(EXECUTE_OUTPUT):
            ! gsutil cat $EXECUTE_OUTPUT
            break
        elif tf.io.gfile.exists(GCP_RESOURCES):
            ! gsutil cat $GCP_RESOURCES
            break

    return EXECUTE_OUTPUT

print("imagedataset-create")
artifacts = print_pipeline_output(pipeline, 'imagedataset-create')
print('\n')
print("automlimagetrainingjob-run")
artifacts = print_pipeline_output(pipeline, 'automlimagetrainingjob-run')
print('\n')
print("endpoint-create")
artifacts = print_pipeline_output(pipeline, 'endpoint-create')
print('\n')
print("model-deploy")
artifacts = print_pipeline_output(pipeline, 'model-deploy')
print('\n')
print("evaluateautomlmodelop")
artifacts = print_pipeline_output(pipeline, 'evaluateautomlmodelop')

### Delete a pipeline job

After a pipeline job is completed, you can delete the pipeline job with the method `delete()`.  Prior to completion, a pipeline job can be canceled with the method `cancel()`.

In [ ]:
pipeline.delete()

# Cleaning up

To clean up all Google Cloud resources used in this project, you can [delete the Google Cloud
project](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects) you used for the tutorial.

Otherwise, you can delete the individual resources you created in this tutorial:

- Dataset
- Pipeline
- Model
- Endpoint
- AutoML Training Job
- Batch Job
- Custom Job
- Hyperparameter Tuning Job
- Cloud Storage Bucket

In [ ]:
delete_all = True

if delete_all:
    # Delete the dataset using the Vertex dataset object
    try:
        if 'dataset' in globals():
            dataset.delete()
    except Exception as e:
        print(e)

    # Delete the model using the Vertex model object
    try:
        if 'model' in globals():
            model.delete()
    except Exception as e:
        print(e)

    # Delete the endpoint using the Vertex endpoint object
    try:
        if 'endpoint' in globals():
            endpoint.undeploy_all()
            endpoint.delete()
    except Exception as e:
        print(e)

    # Delete the AutoML or Pipeline training job
    try:
        if 'dag' in globals():
            dag.delete()
    except Exception as e:
        print(e)

    # Delete the custom training job
    try:
        if 'job' in globals():
            job.delete()
    except Exception as e:
        print(e)

    # Delete the batch prediction job using the Vertex batch prediction object
    try:
        if 'batch_predict_job' in globals():
            batch_predict_job.delete()
    except Exception as e:
        print(e)

    # Delete the hyperparameter tuning job using the Vertex hyperparameter tuning object
    try:
        if 'hpt_job' in globals():
            hpt_job.delete()
    except Exception as e:
        print(e)

    if 'BUCKET_NAME' in globals():
        ! gsutil rm -r $BUCKET_NAME